In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
from airflow import DAG
from airflow.operators.python import PythonOperator


Reading the config from /mnt/HC_Volume_18315164/home-jupyter/jupyter-v-skurlatova-33/airflow/airflow.cfg
Configured default timezone Timezone('UTC')


- Найти топ-10 доменных зон по численности доменов.
- Найти домен с самым длинным именем (если их несколько, то взять только первый в алфавитном порядке).
- На каком месте находится домен airflow.com?

In [2]:

TOP_1M_DOMAINS = 'http://s3.amazonaws.com/alexa-static/top-1m.csv.zip'
TOP_1M_DOMAINS_FILE = 'top-1m.csv'


def get_data():
    top_doms = pd.read_csv(TOP_1M_DOMAINS)
    top_data = top_doms.to_csv(index=False)

    with open(TOP_1M_DOMAINS_FILE, 'w') as f:
        f.write(top_data)

def get_top():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['domain'])
    top_data_df['domain']= top_data_df.domain.apply(lambda x:x.split('.')[-1])
    top_data_df= top_data_df.value_counts().to_frame().reset_index()
    top_data_df=top_data_df['domain'].to_frame()
    top_data_df=top_data_df.head(10)
    with open('top_data_df.csv', 'w') as f:
        f.write(top_data_df.to_csv(index=False, header=False))
                                       
def get_longest_name():
    longest_name = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['domain']) 
    lengths = longest_name['domain'].str.len()
    argmax = np.where(lengths == lengths.max())[0]
    longest_domain=longest_name.iloc[argmax].sort_values('domain') 
    longest_domain=longest_domain.head(1)
    with open('longest_domain.csv', 'w') as f:
        f.write(longest_domain.to_csv(index=False, header=False ))     
 

def find_rank():
    airflow_rank = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank','domain'])
    airflow_rank = airflow_rank.query('domain== "airflow.com"')
    rank = airflow_rank['rank']
    rank = rank.to_frame()
    with open('rank.csv', 'w') as f:
        f.write(rank.to_csv(index=False, header=False ))  
                                       

def print_data(ds):
    with open('top_data_df.csv', 'r') as f:
        top_domain_ex = f.read()
    with open('longest_domain.csv', 'r') as f:
        longest = f.read()
    with open('rank.csv', 'r') as f:
        airflow_r = f.read()    
    date = ds

    print(f'Top 10 domain extensions for date {date}')
    print(top_domain_ex)

    print(f'The longest domain for date {date}')
    print(longest)
    
    print(f'The rank of "airflow.com" domain for date {date}')
    print(airflow_r)



default_args = {
    'owner': 'v.skurlatova',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2023, 4, 13),
}
schedule_interval = '30 14 * * *'

dag = DAG('v-skurlatova-33', default_args=default_args, schedule_interval=schedule_interval)

t1 = PythonOperator(task_id='get_data',
                    python_callable=get_data,
                    dag=dag)

t2 = PythonOperator(task_id='get_top',
                    python_callable=get_top,
                    dag=dag)

t2_x = PythonOperator(task_id='get_longest_name',
                        python_callable=get_longest_name,
                        dag=dag)

t2_xy = PythonOperator(task_id='find_rank',
                        python_callable=find_rank,
                        dag=dag)

t3= PythonOperator(task_id='print_data',
                    python_callable=print_data,
                    dag=dag)

t1 >> [t2, t2_x,t2_xy] >> t3

<Task(PythonOperator): print_data>